<a href="https://colab.research.google.com/github/LEE-I-HUA/antitrust/blob/CHRIS/HN%26FN%26Opinion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# footnote TXT
!pip install PyMuPDF
!pip install --upgrade PyMuPDF

import re # 正則表達式
import os # 提供和作業系統互動的方式
import fitz # prat of MyPDF Library 處理PDF文件

# target_font_size 目標字體大小 , minimum_font_size 最小字體大小
# extract 提取目標文本到目標txt檔案
def extract_text_by_font_size(pdf_path, txt_path, target_font_size=9.0, min_font_size=10.0):
    doc = fitz.open(pdf_path)

    with open(txt_path, "w", encoding="utf-8") as file:
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            blocks = page.get_text("dict")["blocks"]
            extracted_text = f"--- 頁碼 {page_num + 1} ---\n"

            for b in blocks:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            size = span["size"]
                            text = span["text"]

                            # 找出9號字的内容
                            if size == target_font_size:
                                extracted_text += f"{text} "

                            # 找出大於10號字的内容
                            if size > min_font_size:
                                extracted_text += f"{text} "

            file.write(extracted_text + "\n\n")

    doc.close()

# 以不區分大小寫的方式在每個 "關鍵字" 和 "Reference" 中提取文本
def extract_between_keywords(full_text, keywords, output_path):
    patterns = [re.compile(rf"{re.escape(keyword)}.*?Reference", re.DOTALL | re.IGNORECASE) for keyword in keywords]  #Core Terms 或 Reference

    for pattern in patterns:
            matches = pattern.findall(full_text)
            for match in matches:
                file.write(match + "\n\n")

# re. = function from regular expression, complies re pattern into re object for matching operations
# rf = raw format string , creat strings without without print / as escape characters
# escape character為轉義符號 or 跳脫字元 , like /n
# re.escape(keyword) = ensure keyword as literal word form as re
# .*?Reference = match sequence followed by  "Reference"
# re.DOTALL | re.IGNORECASE : flag, DOTALL for matching any character('.' & '/n'). IGNORECASE for match both uppercase & lowercase letter( a & A)
# for keyword in keywords : every keyword work these operation

pdf_file_path = "/content/drive/MyDrive/textmining/StateSupremeCourt.PDF" #無法讀取檔案可以換個路徑就可以了
txt_file_path = "/content/drive/MyDrive/textmining/footnote0109-supreme.txt"

extract_text_by_font_size(pdf_file_path, txt_file_path)

# 關鍵字（大寫）
keywords = ['Syllabus', 'Opinion']

# 讀取
with open(txt_file_path, "r", encoding="utf-8") as file:
    full_text = file.read()



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 43.9 MB/s eta 0:00:00


In [ ]:
# FN CSV
import re
import csv
import fitz

def extract_text_by_font_size(pdf_path, csv_path, target_font_size=9.0, min_font_size=10.0):
    doc = fitz.open(pdf_path)

    with open(csv_path, "w", encoding="utf-8", newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['頁碼', '內容'])

        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            blocks = page.get_text("dict")["blocks"]
            extracted_text = ""

            for b in blocks:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            size = span["size"]
                            text = span["text"]

                            # 找出9號字的内容
                            if size == target_font_size:
                                extracted_text += f"{text} "

                            # 找出大於10號字的内容
                            if size > min_font_size:
                                extracted_text += f"{text} "

            if extracted_text.strip():  # 只有在提取到內容時才寫入 CSV
                csv_writer.writerow([page_num + 1, extracted_text.strip()])

    doc.close()

def extract_between_keywords(full_text, keywords, output_path):
    patterns = [re.compile(rf"{re.escape(keyword)}.*?Reference", re.DOTALL | re.IGNORECASE) for keyword in keywords]  #Core Terms 或 Reference

    with open(output_path, "a", encoding="utf-8") as file:
        for pattern in patterns:
            matches = pattern.findall(full_text)
            for match in matches:
                file.write(match + "\n\n")

# 檔案路徑
pdf_file_path = "/content/drive/MyDrive/textmining/State Surperior Court (CA) (30).PDF"
csv_file_path = "/content/drive/MyDrive/textmining/FN/footnote_Surperior.csv"

# 提取到 CSV 檔案
extract_text_by_font_size(pdf_file_path, csv_file_path)

# 關鍵字（大寫）
keywords = ['Syllabus', 'Opinion']

# 讀取
with open(csv_file_path, "r", encoding="utf-8") as file:
    full_text = file.read()


print(f"完成：{csv_file_path}")


完成：/content/drive/MyDrive/textmining/FN/footnote_Surperior.csv


In [ ]:
# 分界線

Extracted segments are saved in /content/drive/MyDrive/textmining/HN/HN_Circuit.txt


In [ ]:
# HN CSV 整合 (註解)
#Headnotes 及 HN1 之間 會出現抓錯內容的HN編號(手動刪除) 99%抓取正確
# 若只有 Core Terms及Opinion 會抓到Opinion & opinion by的內容(手動刪除)
# !pip install PyMuPDF
# !pip install --upgrade PyMuPDF
import fitz  # PyMuPDF
import re
import csv

# initialize variables to store page content & ongoing Headnotes
def extract_and_process_segments_with_page_number(pdf_file_path, csv_file_path):
    with fitz.open(pdf_file_path) as doc:
        page_content = []  # 儲存頁面和對應內容
        ongoing_hn_label = None # track ongoing HN label of processing HN sections
        ongoing_hn_content = "" # accumulate content of ongoing HN sections

# append "page", "HN label", "content" to page_content list
        for page_num in range(len(doc)):
            page = doc.load_page(page_num) # iterate through each page of PDF
            text = page.get_text()

            # 提取 Core Terms 和 Headnotes

            core_terms_match = re.search(r'Core Terms\s+(.+?)(?=LexisNexis® Headnotes|Counsel|$)', text, re.S)
            # match "Core Terms", /s "match >=1 whitespace characters", (.+?) "capture any character(except newline) times and store.
            # (?=LexisNexis® Headnotes|Counsel|$) : ensure the ending is "Headnotes or Counsel" . $ 結尾為Core Terms字串
            # re.S : flag, makes "." match all character(newline), store in core_terms_match
            if core_terms_match:
                core_terms_content = core_terms_match.group(1).strip() # extracted content between first captured group
                page_content.append([page_num + 1, "Core Terms", core_terms_content])

            headnotes_match = re.search(r'LexisNexis® Headnotes\s+(.+?)(?=HN\d+|$)', text, re.S)
            if headnotes_match:
                headnotes_content = headnotes_match.group(1).strip()
                page_content.append([page_num + 1, "Headnotes", headnotes_content])

            # 提取 HN 部分，包含 “>” 的内容，並處理跨頁問題
            hn_matches = re.findall(r'HN\d+\[|[^.]+>.*?(?=HN\d+|$)|(?<=\>).*?(?=Lawyers\' Counsel or Judges)', text, re.DOTALL)
            # re.findall: find all match of re, above re = find patterns relate to "HN", content with ">", handle spans(跨頁)

            for match in hn_matches: # examine the uncatched "HN content"
                if 'HN' in match:
                    if ongoing_hn_label and ongoing_hn_content:
                        page_content.append([page_num + 1, ongoing_hn_label, ongoing_hn_content.strip()])
                    ongoing_hn_label = match
                    ongoing_hn_content = ""
                elif ">" in match and ongoing_hn_label: # examine whether ">" exist in HN content or not
                    ongoing_hn_content += match.strip() + " "

            # 檢查是否有未完成的HN部分
            if ongoing_hn_label and ongoing_hn_content:
                page_content.append([page_num + 1, ongoing_hn_label, ongoing_hn_content.strip()])
                ongoing_hn_label = None
                ongoing_hn_content = ""

        # 寫入CSV文件
        with open(csv_file_path, 'w', encoding='utf-8', newline='') as csv_file: # "with" ensure closed after writed ,'w' = write mode, newline = '' ensure currect ending
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(['Page', 'Section', 'Content'])
            csv_writer.writerows(page_content)

# 指定的文件路径
pdf_file_path = '/content/drive/MyDrive/textmining/Federal District (30).PDF'
csv_file_path = '/content/drive/MyDrive/textmining/HN/HN_District_wrong.csv'

# 調用函数
extract_and_process_segments_with_page_number(pdf_file_path, csv_file_path)


In [ ]:
# HN CSV 整合
# !pip install PyMuPDF
# !pip install --upgrade PyMuPDF
import fitz  # PyMuPDF
import re
import csv

def extract_and_process_segments_with_page_number(pdf_file_path, csv_file_path):
    with fitz.open(pdf_file_path) as doc:
        page_content = []
        ongoing_hn_label = None
        ongoing_hn_content = ""

        for page_num in range(len(doc)):
            page = doc.load_page(page_num) # iterate through each page of PDF
            text = page.get_text()

            core_terms_match = re.search(r'Core Terms\s+(.+?)(?=LexisNexis® Headnotes|Counsel|$)', text, re.S)

            if core_terms_match:
                core_terms_content = core_terms_match.group(1).strip()
                page_content.append([page_num + 1, "Core Terms", core_terms_content])

            headnotes_match = re.search(r'LexisNexis® Headnotes\s+(.+?)(?=HN\d+|$)', text, re.S)
            if headnotes_match:
                headnotes_content = headnotes_match.group(1).strip()
                page_content.append([page_num + 1, "Headnotes", headnotes_content])


            hn_matches = re.findall(r'HN\d+\[|[^.]+>.*?(?=HN\d+|$)|(?<=\>).*?(?=Lawyers\' Counsel or Judges)', text, re.DOTALL)


            for match in hn_matches:
                if 'HN' in match:
                    if ongoing_hn_label and ongoing_hn_content:
                        page_content.append([page_num + 1, ongoing_hn_label, ongoing_hn_content.strip()])
                    ongoing_hn_label = match
                    ongoing_hn_content = ""
                elif ">" in match and ongoing_hn_label:
                    ongoing_hn_content += match.strip() + " "


            if ongoing_hn_label and ongoing_hn_content:
                page_content.append([page_num + 1, ongoing_hn_label, ongoing_hn_content.strip()])
                ongoing_hn_label = None
                ongoing_hn_content = ""

        with open(csv_file_path, 'w', encoding='utf-8', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(['Page', 'Section', 'Content'])
            csv_writer.writerows(page_content)

pdf_file_path = '/content/drive/MyDrive/textmining/Federal District (30).PDF'
csv_file_path = '/content/drive/MyDrive/textmining/HN/HN_District_wrong.csv'

extract_and_process_segments_with_page_number(pdf_file_path, csv_file_path)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# HN CSV for Surperior
# Surperior 只有 "Core Terms" 的內容 以 Judges or Counsel or Opinion 作為結尾切割

# !pip install PyMuPDF
# !pip install --upgrade PyMuPDF

import fitz  # PyMuPDF
import re
import csv

def extract_core_terms(text):
    # 使用非貪婪匹配提取 Core Terms 內容
    core_terms_match = re.search(r'Core Terms(.*?)(Opinion|Counsel|Judges|$)', text, re.DOTALL)
    if core_terms_match:
        core_terms_content = core_terms_match.group(1).strip()
        return core_terms_content
    else:
        return ""

def extract_and_process_segments_with_page_number(pdf_file_path, csv_file_path):
    with fitz.open(pdf_file_path) as doc:
        page_content = []  # 用於存儲頁面和對應的內容

        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text = page.get_text()

            core_terms_content = extract_core_terms(text)
            if core_terms_content:
                page_content.append([page_num + 1, "Core Terms", core_terms_content])

        # 寫入CSV文件
        with open(csv_file_path, 'w', encoding='utf-8', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(['Page', 'Section', 'Content'])
            csv_writer.writerows(page_content)

# 指定的文件路徑
pdf_file_path = '/content/drive/MyDrive/textmining/State Surperior Court (CA) (30).PDF'
csv_file_path = '/content/drive/MyDrive/textmining/Core_Terms_output_with_page.csv'

# 調用函數
extract_and_process_segments_with_page_number(pdf_file_path, csv_file_path)


In [ ]:
# opinion by , councel,judges,
import fitz
import re
import csv

pdf_file_path = '/content/drive/MyDrive/textmining/StateSupremeCourt.PDF'
csv_file_path = '/content/drive/MyDrive/textmining/opinion_Surpreme.csv'

pattern = r'(Counsel:.*?|Judges:.*?|Opinion by:.*?)(?=Counsel:|Judges:|Opinion by:|Opinion|$)'

with fitz.open(pdf_file_path) as doc:
    with open(csv_file_path, 'w', encoding='utf-8', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['頁碼', '內容'])

        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text = page.get_text("text")

            matches = re.findall(pattern, text, re.DOTALL)

            for match in matches:
                csv_writer.writerow([page_num + 1, match.strip()])

print(f"完成：{csv_file_path}")

# Appeal p.211後面的judges 包含 opinion by 所以摘出來的judges為空白

完成：/content/drive/MyDrive/textmining/opinion_Surpreme.csv


In [ ]:
# 解決Surpreme 層級的 Judges空白問題，（p.809會因為Counsel內容跨頁 而無法抓到跨頁的部分）
import fitz
import re
import csv

pdf_file_path = '/content/drive/MyDrive/textmining/State Appeal Court (CA).PDF'
csv_file_path = '/content/drive/MyDrive/textmining/opinion_Appeal.csv'

pattern = r'(Counsel:.*?|Judges:.*?(?=Opinion by:)|Opinion by:.*?)((?=Counsel:)|(?=Judges:)|(?=Opinion by:)|(?=Opinion)|$)'
# " (Counsel:.*?|Judges:.*?(?=Opinion by:)|Opinion by:.*?) "  :
# capture content of " Counsel: ", " Judges: " 到 "Opinion by:" or "Opinion"

# " ((?=Counsel:)|(?=Judges:)|(?=Opinion by:)|(?=Opinion)|$) " :
# capture the type of the match ("Counsel:", "Judges:", "Opinion by:", "Opinion", or the end of the string).

with fitz.open(pdf_file_path) as doc:
    with open(csv_file_path, 'w', encoding='utf-8', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['頁碼', '內容類型', '內容'])

        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text = page.get_text("text")

            matches = re.finditer(pattern, text, re.DOTALL) # find all occourance of pattern in 'text'
            last_match_type = None # keep track last match type

            for match in matches: #loop initiate matches from re.finditer
                match_content = match.group(1).strip() # retrive re captured content from group1
                match_type = match.group(2).strip()

                if 'Counsel:' in match_content:
                    last_match_type = 'Counsel'
                    csv_writer.writerow([page_num + 1, 'Counsel', match_content])
                elif 'Judges:' in match_content:
                    last_match_type = 'Judges'
                    csv_writer.writerow([page_num + 1, 'Judges', match_content])
                elif 'Opinion by:' in match_content:
                    last_match_type = 'Opinion by'
                    csv_writer.writerow([page_num + 1, 'Opinion by', match_content])
                else:
                    # 若上一個匹配的標記是 Opinion by 且 current type not Judges 則  write Judges' content into csv
                    if last_match_type == 'Opinion by' and 'Judges:' not in match_type:
                        csv_writer.writerow([page_num + 1, 'Judges', match_content])

                    # 檢查下一頁的開頭是否屬於這個匹配的結尾, 看matches後面是否有更多content, 將next 20 character start with Counsel to be "None"
                    if match.end() < len(text) and text[match.end():match.end()+20].strip().startswith('Counsel:'):
                        last_match_type = None  # 現在的內容跨頁了

print(f"完成：{csv_file_path}")


In [ ]:
# HN TXT 提取

In [ ]:
# HN TXT STEP1
import fitz  # PyMuPDF
import re

def extract_segments(pdf_file_path, txt_file_path):
    # 打開PDF檔案
    with fitz.open(pdf_file_path) as doc:
        text = ""
        for page in doc:
            text += page.get_text()

    start_marker = 'Core Terms'
    end_marker = 'Counsel' or 'Judges'
    pattern = re.compile(re.escape(start_marker) + '.*?' + re.escape(end_marker), re.DOTALL)

    matches = pattern.findall(text)

    with open(txt_file_path, 'w') as file:
        for match in matches:
            file.write(match + '\n\n-----\n\n')

    return txt_file_path
extracted_file = extract_segments('/content/drive/MyDrive/textmining/Federal Circuit (30).PDF', '/content/drive/MyDrive/textmining/HN/HN_Circuit.txt')

print(f"Extracted segments are saved in {extracted_file}")

Extracted segments are saved in /content/drive/MyDrive/textmining/HN/HN_Circuit.txt


In [ ]:
# HN TXT STEP 2
import re

txt_input_path = '/content/drive/MyDrive/textmining/HN/HN_Circuit.txt'
txt_output_path = '/content/drive/MyDrive/textmining/HN/HN_Circuit_output.txt'

with open(txt_input_path, 'r') as file:
    full_text = file.read()

matches1 = re.findall(r'HN\d+\[|[^.]+>.*?(?=HN\d+)|(?<=\>).*?(?=Lawyers\' Counsel or Judges)', full_text, re.DOTALL)
cleaned_matches = [re.sub(r'Page \d+ of \d+', '', match) for match in matches1]

with open(txt_output_path, 'w') as txt_file:
    for match in cleaned_matches:
        txt_file.write(match.strip() + "\n\n")

print(f"完成：{txt_output_path}")

完成：/content/drive/MyDrive/textmining/HN/HN_Circuit_output.txt
